```
Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.
e.g.: Import the dataset with :
                                "mongoimport --type csv -d SP500 -c SP500_ESG --headerline --drop sp500_esg_ceo_info-filtered.csv"
                                "mongoimport --type csv -d SP500 -c date --headerline --drop datefiltered_data.csv"
```

In [4]:
# Import dependencies
import pandas as pd
from pymongo import MongoClient
from pprint import pprint
import seaborn as sns



In [5]:
# Create an instance of MongoClient
mongo = MongoClient(port = 27017)

In [7]:
# Confirm that our new database was created
MongoDB = mongo.list_database_names()
print(MongoDB)

['SP500', 'admin', 'autosaurus', 'classDB', 'config', 'epa', 'fruits_db', 'gardenDB', 'local', 'petsitly_marketing', 'travel_db', 'uk_food']


In [8]:
# Assign the uk_food database to a variable name
db = mongo["SP500"]

In [9]:
# Review the collections in our new database
collections = db.list_collection_names()
print(collections)

['date', 'SP500_ESG']


In [10]:
# Fetch data from collections
date = db["SP500"]
SP500_ESG = db["SP500"]

In [20]:
# Rename Symbol fiel in date collection
db.date.update_many({}, {"$rename": {"Symbol": "Ticker"}})

print("Field 'Symbol' renamed to 'Ticker' successfully!")

Field 'Symbol' renamed to 'Ticker' successfully!


In [31]:
# Merge data from 'date' and 'SP500_ESG' collections
result = db.SP500_ESG.aggregate([
    {
        "$lookup": {
            "from": "date",
            "localField": "Ticker",
            "foreignField": "Ticker",
            "as": "merged_data"
        }
    }
])

# Create a new collection and insert the merged data
merged_collection = db["sp500_esg_date"]
for doc in result:
    merged_collection.insert_one(doc)

print("Merged data inserted into 'merged_collection' successfully!")

Merged data inserted into 'merged_collection' successfully!


In [32]:
# Fetch data from the collection
cursor = merged_collection.find({})
data = list(cursor)

# Create a DataFram
df = pd.DataFrame(data)

# Save the DataFrame to a CSV file
df.to_csv("sp500_esg_date.csv", index = False)